In [19]:
import os
import torch
import torchvision
import torch.nn as nn
from PIL import Image
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [20]:
data_transform = {
    "train": transforms.Compose([transforms.RandomResizedCrop(224),  # 将数据集图片放缩成224*224大小
                                 transforms.RandomHorizontalFlip(),  # 图片有默认0.5的概率进行垂直旋转

                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
    "test": transforms.Compose([transforms.Resize((224, 224)),  # 调整图片大小为224*224
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}

In [21]:
train_dataset = torchvision.datasets.ImageFolder(root=r"C:\Users\dell\Rice Leaf Disease Images\train",
                                                 transform=data_transform["train"])
print(train_dataset.class_to_idx)

{'Bacterialblight': 0, 'Blast': 1, 'Brownspot': 2, 'Tungro': 3}


In [22]:
test_dataset = torchvision.datasets.ImageFolder(root=r"C:\Users\dell\Rice Leaf Disease Images\test",
                                                transform=data_transform["test"])
print(test_dataset.class_to_idx)

{'Bacterialblight': 0, 'Blast': 1, 'Brownspot': 2, 'Tungro': 3}


In [23]:
train_data = DataLoader(dataset=train_dataset, batch_size=20,  # 将训练数据以每次20张图片的形式抽出进行训练
                        shuffle=True, num_workers=0)
test_data = DataLoader(dataset=test_dataset, batch_size=10,    # 将测试数据以每次10张图片的形式抽出进行测试
                       shuffle=True, num_workers=0)

In [24]:
class BasicBlock(nn.Module):
    expansion = 1 

    def __init__(self, in_channel, out_channel, stride=1, down_sample=None):
        super(BasicBlock, self).__init__()

 
        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=out_channel,
                               kernel_size=3, stride=stride, padding=1, bias=False)

        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU()

        self.conv2 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel,
                               kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)
        self.down_sample = down_sample  # 定义下采样方法=传入的下采样参数
 
    def forward(self, x):                # 正向传播过程，x为输入的特征矩阵
        identity = x                     # 将x赋值给分支identity
        if self.downsample is not None:  
            identity = self.downsample(x)
 

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
 
        out = self.conv2(out)
        out = self.bn2(out)
 

        out += identity
        out = self.relu(out)
 
        return out

In [25]:
class Bottleneck(nn.Module): 

    expansion = 4
 
    def __init__(self, in_channel, out_channel, stride=1, down_sample=None):
        super(Bottleneck, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=out_channel,
                               kernel_size=1, stride=1, bias=False)  # squeeze channels
        self.bn1 = nn.BatchNorm2d(out_channel)

        self.conv2 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel,
                               kernel_size=3, stride=stride, bias=False, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channel)
        self.conv3 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel * self.expansion,
                               kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channel * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.down_sample = down_sample
 

    def forward(self, x):
        identity = x

        if self.down_sample is not None:
            identity = self.down_sample(x)
 
        out = self.conv1(x)   # 卷积层
        out = self.bn1(out)   # BN层
        out = self.relu(out)  # 激活层
 
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
 
        out = self.conv3(out)
        out = self.bn3(out)
 
        out += identity
        out = self.relu(out)
        
        return out

In [26]:
class ResNet(nn.Module):

    def __init__(self, block, blocks_num, num_classes=1000, include_top=True):
        super(ResNet, self).__init__()
        self.include_top = include_top
        self.in_channel = 64  
 

        self.conv1 = nn.Conv2d(3, self.in_channel, kernel_size=7, stride=2,
                               padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channel)
        self.relu = nn.ReLU(inplace=True)
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)  # 对应3*3那个maxpooling

        self.layer1 = self._make_layer(block, 64, blocks_num[0])
        # conv3_x
        self.layer2 = self._make_layer(block, 128, blocks_num[1], stride=2)
        # conv4_x
        self.layer3 = self._make_layer(block, 256, blocks_num[2], stride=2)
        # conv5_x
        self.layer4 = self._make_layer(block, 512, blocks_num[3], stride=2)
        if self.include_top:

            self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))  # output size = (1, 1)

            self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

    def _make_layer(self, block, channel, block_num, stride=1):
        down_sample = None  # 下采样赋值为none

        if stride != 1 or self.in_channel != channel * block.expansion:
            # 生成下采样函数
            down_sample = nn.Sequential(
                nn.Conv2d(self.in_channel, channel * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(channel * block.expansion))
 
        layers = []

        layers.append(block(self.in_channel, channel, down_sample=down_sample, stride=stride))
        self.in_channel = channel * block.expansion
 

        for _ in range(1, block_num):  

            layers.append(block(self.in_channel, channel))

        return nn.Sequential(*layers)
 

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.max_pool(x)
 
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
 
        if self.include_top:
            x = self.avg_pool(x)     
            x = torch.flatten(x, 1)  
            x = self.fc(x)           
        return x

In [27]:
def resnet18(num_classes=1000, include_top=True):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes, include_top=include_top)

In [28]:
def resnet34(num_classes=1000, include_top=True):
    return ResNet(BasicBlock, [3, 4, 6, 3], num_classes=num_classes, include_top=include_top) 
def resnet50(num_classes=1000, include_top=True):
    return ResNet(Bottleneck, [3, 4, 6, 3], num_classes=num_classes, include_top=include_top)
def resnet101(num_classes=1000, include_top=True):
    return ResNet(Bottleneck, [3, 4, 23, 3], num_classes=num_classes, include_top=include_top)
def resnet152(num_classes=1000, include_top=True):
    return ResNet(Bottleneck, [3, 8, 36, 3], num_classes=num_classes, include_top=include_top)

In [29]:
net = resnet152()
model_weight_path = "./resnet152-b121ed2d.pth"  
assert os.path.exists(model_weight_path), "file {} does not exist.".format(model_weight_path)
missing_keys, unexpected_keys = net.load_state_dict(torch.load(model_weight_path), strict=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device) 

epoch = 1                                           
learning_rate = 0.001                       
Loss_func = nn.CrossEntropyLoss()                   
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)  

In [30]:
for epoch in range(epoch):
    train_num = 0     # 训练集样本总数初始设为0
    net.train()  # 设置模型为训练模式，保证 BN层 能够用到每一批数据的均值和方差
    for step, (data, target) in enumerate(train_data):
        data, target = data.to(device), target.to(device)  # 将取出的数据加载到可利用的GPU上
        optimizer.zero_grad()                              # 清空每次循环训练的历史梯度，防止梯度累加而造成结果不收敛
        output = net(data)                      # 将数据传入模型，通过前向传播获得预测值:
        loss_value = Loss_func(output, target)  # 获取损失值（即计算神经网络的输出结果output与图片真实标签target的差别）
        loss_value.backward()        # 进行反向传播：
        optimizer.step()             # 梯度下降更新参数：
        train_num += data.size(0)    # 统计训练集样本总数

        print(f'当前为第{epoch + 1}次循环 '
              f'[此次训练已进行：{step * len(data)}/{len(train_data.dataset)} '
              f'{100 * step / len(train_data):.0f}%)]\tLoss is {loss_value.item():.4f}')
    print()

    test_loss = 0.0   # 测试集的损失值初始设为0
    test_acc = 0.0    # 测试集的准确率初始设为0
    test_num = 0      # 测试集样本总数初始设为0
    accuracy = 0.0
    net.eval()        # 将模型调整为测试模型
    with torch.no_grad():  # 清空历史梯度
        for data, target in test_data:
            data, target = data.to(device), target.to(device)
            output = net(data)
            loss_value2 = Loss_func(output, target)
            _, predicted = torch.max(output.data, 1)    # 得到每行最大值（dim=1,按行进行计算）与每个最大值对应的索引值：
 
            test_loss += loss_value2.item()                   # 每次测试得到的损失值相加
            test_num += data.size(0)                          # 统计样本总数
            accuracy += (predicted == target).sum().item()    # 统计经过模型测试正确的样本个数：
 
        test_acc = 100 * accuracy / test_num                  # 计算测试集的本次循环测试后的总准确率
 
    print(f'第{epoch+1}次循环测试：'
          f'模型损失值为：{test_loss / test_num:.4f} \t 模型准确率为：{test_acc:.4f}%')
    print()

当前为第1次循环 [此次训练已进行：0/4700 0%)]	Loss is 9.4621
当前为第1次循环 [此次训练已进行：20/4700 0%)]	Loss is 3.5677
当前为第1次循环 [此次训练已进行：40/4700 1%)]	Loss is 3.6746
当前为第1次循环 [此次训练已进行：60/4700 1%)]	Loss is 2.8925
当前为第1次循环 [此次训练已进行：80/4700 2%)]	Loss is 1.4804
当前为第1次循环 [此次训练已进行：100/4700 2%)]	Loss is 3.4017
当前为第1次循环 [此次训练已进行：120/4700 3%)]	Loss is 2.9084
当前为第1次循环 [此次训练已进行：140/4700 3%)]	Loss is 2.1183
当前为第1次循环 [此次训练已进行：160/4700 3%)]	Loss is 2.4724
当前为第1次循环 [此次训练已进行：180/4700 4%)]	Loss is 1.8636
当前为第1次循环 [此次训练已进行：200/4700 4%)]	Loss is 1.0518
当前为第1次循环 [此次训练已进行：220/4700 5%)]	Loss is 1.9338
当前为第1次循环 [此次训练已进行：240/4700 5%)]	Loss is 0.8030
当前为第1次循环 [此次训练已进行：260/4700 6%)]	Loss is 0.7191
当前为第1次循环 [此次训练已进行：280/4700 6%)]	Loss is 0.6307
当前为第1次循环 [此次训练已进行：300/4700 6%)]	Loss is 0.7862
当前为第1次循环 [此次训练已进行：320/4700 7%)]	Loss is 1.0930
当前为第1次循环 [此次训练已进行：340/4700 7%)]	Loss is 0.6446
当前为第1次循环 [此次训练已进行：360/4700 8%)]	Loss is 0.6676
当前为第1次循环 [此次训练已进行：380/4700 8%)]	Loss is 1.0893
当前为第1次循环 [此次训练已进行：400/4700 9%)]	Loss is 1.1993
当前为第1次循环 [此次训练已进行：4

当前为第1次循环 [此次训练已进行：3380/4700 72%)]	Loss is 0.5295
当前为第1次循环 [此次训练已进行：3400/4700 72%)]	Loss is 0.3754
当前为第1次循环 [此次训练已进行：3420/4700 73%)]	Loss is 0.6416
当前为第1次循环 [此次训练已进行：3440/4700 73%)]	Loss is 0.8882
当前为第1次循环 [此次训练已进行：3460/4700 74%)]	Loss is 0.8922
当前为第1次循环 [此次训练已进行：3480/4700 74%)]	Loss is 0.5009
当前为第1次循环 [此次训练已进行：3500/4700 74%)]	Loss is 0.8975
当前为第1次循环 [此次训练已进行：3520/4700 75%)]	Loss is 0.4339
当前为第1次循环 [此次训练已进行：3540/4700 75%)]	Loss is 0.6826
当前为第1次循环 [此次训练已进行：3560/4700 76%)]	Loss is 0.4543
当前为第1次循环 [此次训练已进行：3580/4700 76%)]	Loss is 0.3315
当前为第1次循环 [此次训练已进行：3600/4700 77%)]	Loss is 0.4676
当前为第1次循环 [此次训练已进行：3620/4700 77%)]	Loss is 0.4845
当前为第1次循环 [此次训练已进行：3640/4700 77%)]	Loss is 0.3605
当前为第1次循环 [此次训练已进行：3660/4700 78%)]	Loss is 0.1950
当前为第1次循环 [此次训练已进行：3680/4700 78%)]	Loss is 0.6590
当前为第1次循环 [此次训练已进行：3700/4700 79%)]	Loss is 0.5262
当前为第1次循环 [此次训练已进行：3720/4700 79%)]	Loss is 0.8428
当前为第1次循环 [此次训练已进行：3740/4700 80%)]	Loss is 0.3157
当前为第1次循环 [此次训练已进行：3760/4700 80%)]	Loss is 0.2936
当前为第1次循环 [此次训练已进行：37

In [32]:
torch.save(net,'./shuidao.pth')

C:\Users\dell\.conda\envs\tf\lib\site-packages\torch\serialization.py:402: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\dell\.conda\envs\tf\lib\site-packages\torch\serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Bottleneck. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [34]:
class_names = ['Bacterialblight——白枯病', 'Blast——稻瘟病', 'Brownspot——褐斑病', 'Tungro——钨腐病']
model = torch.load('shuidao.pth')
model.to(device)
image = Image.open(r'C:\Users\dell\Rice Leaf Disease Images\train\Blast\BLAST1_121.JPG')
trans = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
image = image.convert("RGB")
image = trans(image)
image = torch.unsqueeze(image, dim=0)

model.eval()             
with torch.no_grad():    
    outputs = model(image.to(device))              
    ans = outputs.argmax(1).clone().detach().item()

In [35]:
print(class_names[ans])

Blast——稻瘟病
